# Don't Start Here

In [5]:
import json
import csv

In [6]:
bounding_school_path = 'bounding_school.json'

In [7]:
with open(bounding_school_path, encoding='utf8') as read_file:
    bounding_school = json.load(read_file)

TypeError: 'encoding' is an invalid keyword argument for this function

In [4]:
bounding_school = bounding_school['elements']

NameError: name 'bounding_school' is not defined

In [16]:
# bounding_local_path = 'bounding_local.json'
# bounding_university_path = 'bounding_university.json'
# bounding_college_path = 'bounding_college.json'

In [3]:
# with open(bounding_local_path, encoding='utf8') as read_file:
#     bounding_local = json.load(read_file)
# with open(bounding_university_path, encoding='utf8') as read_file:
#     bounding_university = json.load(read_file)
# with open(bounding_college_path, encoding='utf8') as read_file:
#     bounding_college = json.load(read_file)

In [4]:
# bounding_local = bounding_local['elements']
# bounding_university = bounding_university['elements']
# bounding_college = bounding_college['elements']

In [5]:
# really expensive dont run, done to find out just how many repeats there are and their format
# repeats = []
# hasseen = []
# mapping = {}
# for j, bound in enumerate(bounding_global):
#     #print(float(j) / float(len(bounding_global)))
#     aid = bound['id']
#     if aid in hasseen:
#         repeats.append(aid)
#     else:
#         hasseen.append(aid)

In [6]:
# for j, bound in enumerate(bounding_global):
#     if repeats[447] == bound['id']:
#         print(bound)
# #     for repeat in repeats:
# #         if repeat[0] == bound['id']:
# #             print(bound)

In [7]:
def changetoidbaseddict(boundings):
    iddict = {}
    counterforrepeat = 0
    for j, bound in enumerate(boundings):
        idinquestion = str(bound['id'])
        #if already in the dictionary
        if idinquestion in iddict:
            #if the new element has more keys update it
            if len(bound.keys()) > len(iddict[idinquestion].keys()):
                print(counterforrepeat)
                counterforrepeat += 1
                iddict[idinquestion] = bound
        else:
            iddict[idinquestion] = bound
    return iddict

In [8]:
bounding_local = changetoidbaseddict(bounding_local)
bounding_university = changetoidbaseddict(bounding_university)
bounding_college = changetoidbaseddict(bounding_college)

In [9]:
def format_openapi(bounding):
    coordinatelists = []
    usedids = set()
    relationcount = 0
    waycount = 0
    nodecount = 0
    for boundkey in bounding:
        #reset lists
        bound = None
        waylist = []
        nodelist = []
        relname = None
        nameisid = None
        eachwaymember = None
        potentialwaymember = None
        nodeid = None
        wayid = None
        thenode = None
        theway = None
        allnodesofway = None
        
        #initialize item of dictionary from key
        bound = bounding[boundkey]
        
        #if it is a relation and the relation key is not in the set
        if 'relation' == bound['type'] and not boundkey in usedids:
            
            nameisid = True
            
            waylist = []
            
            #append initial id to used ids otherwise
            usedids.add(str(boundkey))
            
            relationcount += 1
            
            relname = str(boundkey)
            if 'tags' in bound and 'name' in bound['tags']:
                relname = bound['tags']['name']
                nameisid = False
                
            if not 'members' in bound:
                print("relation " + boundkey + " had no members")
                continue
                
            waymembers = bound['members']
            for potentialwaymember in waymembers:
                eachwaymember = None
                #reset nodelist on every passthrough
                nodelist = []
                
                if 'way' != potentialwaymember['type']:
                    if 'node' == potentialwaymember['type']:
                        nodeid = potentialwaymember['ref']
                        if nodeid in usedids:
                            print("reusedkey " + str(nodeid))
                            return None

                        #append the node from the way from the relation to used ids
                        usedids.add(str(nodeid))
                        
                        nodecount += 1

                        #id of the node from the way in the loop
                        thenode = bounding[str(nodeid)]

                        #append the latitude longitude from the node in a tuple 
                        nodelist.append((thenode['lat'], thenode['lon']))
                        
                        thenode = None
                        continue
                    elif 'relation' == potentialwaymember['type']:
                        print("relation " + boundkey + " with relation " + str(potentialwaymember['ref']) + " has no id")
                        continue
                    else:
                        print("relation " + boundkey + " had a member " + str(potentialwaymember['ref']) + " that did not fit a type")
                        continue
                else:
                    eachwaymember = potentialwaymember
                
                #id of current way
                wayid = eachwaymember['ref'] 
                
                #if the way from the relation has been used
                if wayid in usedids:
                    print("relation " + boundkey + " reused way " + str(wayid))
                    return None
                
                #append way id evaluated to used ids otherwise
                usedids.add(str(wayid))
                
                waycount += 1
                
                #actual way being evaluated
                theway = bounding[str(wayid)]
                
                if not 'nodes' in theway:
                    #assume the way includes a latitude and longitude itself throw if not
                    if 'lat' in theway and 'lon' in theway:  
                        nodelist.append((theway['lat'], theway['lon']))
                        print("**relation" + str(boundkey) + " with way " + str(wayid) + " with no lat lon in way itself")
                    continue
                #all nodes of the way in the loop
                allnodesofway = theway['nodes']
                for nodeid in allnodesofway:
                    
                    #if the way from the relation has been used
                    if nodeid in usedids:
                        print("reusedkey " + str(nodeid))
                        return None
                    
                    #append the node from the way from the relation to used ids
                    usedids.add(str(nodeid))
                    
                    nodecount += 1
                    
                    #id of the node from the way in the loop
                    thenode = bounding[str(nodeid)]
                    
                    #append the latitude longitude from the node in a tuple
                    if 'lat' in thenode and 'lon' in thenode: 
                        nodelist.append((thenode['lat'], thenode['lon']))
                    else:
                        print("##relation " + str(boundkey) + " with node " + str(nodeid) + " has no lat lon")
                if nodelist:
                    #if the nodelist isnt empty append the nodeset to the wayset
                    waylist.append(nodelist)
            if waylist:       
                #if the wayset isnt empty create the tempdict and append it to the final mapping and the list
                tempdict = {}
                tempdict['name'] = relname
                tempdict['type'] = 'relation'
                tempdict['nameisid'] = nameisid
                tempdict['nodelist'] = waylist
                coordinatelists.append(tempdict) 
    for boundkey in bounding:
        #reset lists
        bound = None
        waylist = []
        nodelist = []
        relname = None
        nameisid = None
        bound = bounding[boundkey]
        allnodesofway = None
        nodeid = None
        thenode = None
        
        if 'way' == bound['type'] and not boundkey in usedids:
            nameisid = True
            nodelist = []

            #append initial id to used ids otherwise
            usedids.add(str(boundkey))
            
            waycount += 1
            
            relname = str(boundkey)
            if 'tags' in bound and 'name' in bound['tags']:
                nameisid = False
                relname = bound['tags']['name']
            
            allnodesofway = bound['nodes']
            
            for nodeid in allnodesofway:

                #if the way from the relation has been used
                if nodeid in usedids:
                    print("reusedkey " + str(nodeid))
                    return None

                #append the node from the way from the relation to used ids
                usedids.add(str(nodeid))
                
                nodecount += 1

                #id of the node from the way in the loop
                thenode = bounding[str(nodeid)]

                #append the latitude longitude from the node in a tuple
                if 'lat' not in thenode or 'lon' not in thenode:
                    print("relation " + boundkey + " with lat or lon not in node " + str(nodeid))
                else:
                    nodelist.append((thenode['lat'], thenode['lon']))
            if nodelist:
                #if the nodelist isnt empty append the nodeset to the wayset
                tempdict = {}
                tempdict['name'] = relname
                tempdict['type'] = 'way'
                tempdict['nameisid'] = nameisid
                tempdict['nodelist'] = nodelist
                coordinatelists.append(tempdict)
     
    print("viable relations: "  + str(relationcount))
    print("viable ways: "  + str(waycount))
    print("viable nodes: "  + str(nodecount))
    return coordinatelists

In [10]:
bounding_university_parsed = format_openapi(bounding_university)
bounding_college_parsed = format_openapi(bounding_college)
bounding_local_parsed = format_openapi(bounding_local)

relation 9129 with relation 1111570 has no id
relation 31489 with relation 3334678 has no id
relation 31489 with relation 3334680 has no id
relation 31489 with relation 2906997 has no id
relation 31489 with relation 2912378 has no id
relation 31489 with relation 3431722 has no id
relation 31489 with relation 3431721 has no id
relation 31489 with relation 3293414 has no id
relation 31489 with relation 963101 has no id
relation 31489 with relation 452374 has no id
relation 31489 with relation 452370 has no id
relation 33319 with relation 2649793 has no id
relation 33319 with relation 2650012 has no id
relation 33319 with relation 2650830 has no id
relation 34214 with relation 1318220 has no id
relation 34214 with relation 34213 has no id
relation 34214 with relation 6043642 has no id
relation 36798 with relation 3246522 has no id
relation 36798 with relation 3246423 has no id
relation 36798 with relation 3246422 has no id
relation 130868 with relation 31533 has no id
relation 1070152 wit

relation 8710692 with relation 8710691 has no id
relation 8727202 with relation 8755153 has no id
relation 33197031 with lat or lon not in node 376316209
relation 468662728 with lat or lon not in node 599059068
viable relations: 2656
viable ways: 46953
viable nodes: 795606
relation 531720 had no members
relation 1971906 had no members
relation 2437216 had no members
relation 2559343 had no members
relation 2740971 had no members
relation 2885924 had no members
relation 2955467 had no members
relation 2993255 had no members
relation 3030392 had no members
relation 3151882 had no members
relation 3368293 with relation 3368292 has no id
relation 3368293 with relation 3368291 has no id
relation 3368293 with relation 3368290 has no id
relation 3368293 with relation 3368289 has no id
relation 3382879 had no members
relation 3438264 had no members
##relation 3757004 with node 428216384 has no lat lon
relation 3861680 had no members
relation 3936004 had no members
relation 4557497 had no membe

In [11]:
def removeabsoluteduplicates(seq): 
   # order preserving
   noDupes = []
   [noDupes.append(i) for i in seq if not noDupes.count(i)]
   return noDupes

In [12]:
bounding_university_parsed = removeabsoluteduplicates(bounding_university_parsed)
bounding_college_parsed = removeabsoluteduplicates(bounding_college_parsed)
bounding_local_parsed = removeabsoluteduplicates(bounding_local_parsed)

In [13]:
def checkduplicatenamesbetweenlists(original, comparison):
    namecheck = {}
    for key in original:
        coordinatelocation = original[key]
        if 'relation' == coordinatelocation['type'] and 'tags' in coordinatelocation and 'name' in coordinatelocation['tags']:
            thename = coordinatelocation['tags']['name']
            namecheck[thename] = False
        if 'way' == coordinatelocation['type'] and 'tags' in coordinatelocation and 'name' in coordinatelocation['tags']:
            thename = coordinatelocation['tags']['name']
            namecheck[thename] = False

    for coordinatelocation in comparison:
        thename = coordinatelocation['name']
        if thename in namecheck:
            namecheck[thename] = True

    for name in namecheck:
        if namecheck[name] == False:
            print(name)

In [15]:
checkduplicatenamesbetweenlists(bounding_university, bounding_university_parsed)
checkduplicatenamesbetweenlists(bounding_college, bounding_college_parsed)

Media Humanities and Technology
Technische Fakultät
Medizinisch-Naturwissenschaftliches Forschungszentrum
UBO - UFR Médecine
Curtin University (Bentley Campus)
TU Darmstadt / S1
UQÀM - Complexe des sciences Pierre-Dansereau
Отдел снабжения СПбГПУ
TU Darmstadt / S2
Université de Bretagne-Occidentale
Fern-Universität Hagen
Université de Rennes 1 - Campus de Beaulieu
Anorganische Chemie
Entsorgungszentrum
Physikalische Chemie
Clemens-Schöpf-Institut
Materialwissenschaft
Duke University West Campus
Campus Tal
Institut für Ethik und Geschichte in der Medizin
Anatomisches Institut (Alte Anatomie)
Campus Sand
Sonderforschungsbereich 666, Prozesslernfabrik
Alte Aula
Alte Burse / Institut für Philosophie und Kunstgeschichte
Kekulé-Auditorium
San José State University Main Campus
Arboretum des Botanischen Gartens
Maschinenbauhalle 1
Maschinenbauhalle 5
Forschungsgebäude 1
Bauingenieurhalle 2
Bauingenieurhalle 3
Bauingenieurhalle 4
Bauingenieurwesen Instiutsgebäude 2
Campus Hohbuch
Informática
Fa

Castlereagh Campus
Birmingham Metropolitan College James Watt Campus
École normale supérieure – Campus de Jourdan
Hochschule Biberach
The Woodbury Centre
Millfield Campus
École normale supérieure – Campus d'Ulm
École normale supérieure – Annexe
INSA Centre Val de Loire - Bâtiment Jean Laigret
INSA Centre Val de Loire - Hall technologique
INSA Centre Val de Loire - Bâtiment Gambetta
Birmingham Metropolitan College (Sutton Coldfield Campus)
Institución Educativa Venedikta Zur Nieden
Blackpool and the Fylde College
Sede Centrale
The Enterprise Centre
Sede Nievo A
Sede Nievo B
The Venture Centre
ПТШ N10
UPLB College of Arts and Sciences
Norco College
École de la Nature et du Paysage
Institut National des Sciences Appliquées - Centre Val de Loire - Campus de Blois
gibb Gewerblich-Industrielle Berufsschule Bern
Springvale Campus
Sede Fasolo
ГРП
Titanic Quarter Campus
南校区
北校区
Корпус № 2 МДПК
I.E.S. de Ponteareas
IES Carlos Casares
Коледж "Электроники"
Московский Приборостроительный Техникум
П

In [17]:
# with open(bounding_university_path[:-5] + '_parsed.json', 'w') as write_file:
#     json.dump(bounding_university_parsed, write_file, indent=4)
# with open(bounding_college_path[:-5] + '_parsed.json', 'w') as write_file:
#     json.dump(bounding_college_parsed, write_file,indent=4)
# with open(bounding_local_path[:-5] + '_parsed.json', 'w') as write_file:
#     json.dump(bounding_local_parsed, write_file,indent=4)

# Start Here With Updated Jsons

In [1]:
import json

In [2]:
bounding_local_parsed_path = 'bounding_local_parsed.json'
bounding_university_parsed_path = 'bounding_university_parsed.json'
bounding_college_parsed_path = 'bounding_college_parsed.json'

In [3]:
with open(bounding_local_parsed_path, 'r') as read_file:
    bounding_local = json.load(read_file)
with open(bounding_university_parsed_path, 'r') as read_file:
    bounding_university = json.load(read_file)
with open(bounding_college_parsed_path, 'r') as read_file:
    bounding_college = json.load(read_file)

In [29]:
def searchforname(searchname, inlist):
    for coordinatelocation in inlist:
        thename = coordinatelocation['name']
        if searchname in thename or searchname.lower() in thename:
            print(coordinatelocation)

In [30]:
searchforname('University of California, Berkeley', bounding_university)

{'name': 'University of California, Berkeley', 'type': 'relation', 'nameisid': False, 'nodelist': [[[37.8741433, -122.2673891], [37.8742417, -122.2664637], [37.8746595, -122.2665204], [37.875919, -122.2666733], [37.8767927, -122.2667689], [37.8767232, -122.2673536], [37.8766813, -122.267706], [37.8764378, -122.2676768], [37.875884, -122.2676159], [37.8741433, -122.2673891]], [[37.8814193, -122.2318091], [37.8813177, -122.2315345], [37.8811348, -122.2311225], [37.8808909, -122.2308049], [37.8807758, -122.2304616], [37.8801999, -122.2296633], [37.8797934, -122.2286763], [37.8792853, -122.2277322], [37.879116, -122.2272601], [37.8781336, -122.2264447], [37.8776255, -122.2254576], [37.8774223, -122.224771], [37.8767448, -122.2234835], [37.8761351, -122.2223677], [37.8756269, -122.2218098], [37.8750511, -122.2221532], [37.8746445, -122.2229686], [37.8743058, -122.2247281], [37.8733911, -122.2271313], [37.872883, -122.2270455], [37.8724426, -122.2275605], [37.8721038, -122.2279467], [37.8722

In [6]:
def checkduplicatenamesinsamelist(searchlist):
    nameset = {}
    for bound in searchlist:
        thename = bound['name']
        if thename in nameset:
            if nameset[thename] == bound:
                print(bound)
        else:
            nameset[thename] = bound

In [7]:
checkduplicatenamesinsamelist(bounding_university)

In [8]:
def arbitraryflatten(iterable):
    iterator, sentinel, stack = iter(iterable), object(), []
    while True:
        value = next(iterator, sentinel)
        if value is sentinel:
            if not stack:
                break
            iterator = stack.pop()
        elif isinstance(value, str):
            yield value
        else:
            try:
                new_iterator = iter(value)
            except TypeError:
                yield value
            else:
                stack.append(iterator)
                iterator = new_iterator

In [26]:
def checklatlonvaluedifference(searchlist):
    for bound in searchlist:
        flattened = list(arbitraryflatten(bound['nodelist']))
        latitudes = [element for i, element in enumerate(flattened) if i%2==0]
        longitudes = [element for i, element in enumerate(flattened) if i%2!=0]
        resultslat = any([abs(x - latitudes[i+1]) > 0.007 for i, x in enumerate(latitudes) if i != len(latitudes) - 1])
        resultslon = any([abs(x - longitudes[i+1]) > 0.007 for i, x in enumerate(longitudes) if i != len(longitudes) - 1])
        if resultslat or resultslon:
            print(bound)
    

In [27]:
checklatlonvaluedifference(bounding_college)

{'name': 'Chaffey College', 'type': 'relation', 'nameisid': False, 'nodelist': [[[34.1503841, -117.5756026], [34.1504003, -117.5756005], [34.1504173, -117.5755958], [34.1504322, -117.5755855], [34.150448, -117.5755737], [34.150462, -117.5755559], [34.1504719, -117.5755342], [34.1504784, -117.5755083], [34.1504798, -117.5754764], [34.1505349, -117.5654644], [34.1505065, -117.5651726], [34.1434742, -117.5651898], [34.1434529, -117.5674128], [34.1435523, -117.5684599], [34.1435452, -117.5721678], [34.1434955, -117.5725455], [34.1431403, -117.5734381], [34.1433392, -117.5735325], [34.1433889, -117.5736183], [34.1434031, -117.5737042], [34.143396, -117.5751376], [34.1435448, -117.5756938], [34.1458936, -117.575656], [34.1503841, -117.5756026]], [[34.1458124, -117.5748774], [34.1458046, -117.575261], [34.1457958, -117.575734], [34.1457969, -117.5759024]], [[34.1458124, -117.5748774], [34.1458194, -117.5743316], [34.145815, -117.5741952], [34.1457919, -117.5740815], [34.145759, -117.5739872],

{'name': 'Магілёўскі дзяржаўны політэхнічны каледж', 'type': 'relation', 'nameisid': False, 'nodelist': [[[53.9187298, 30.3428209], [53.9187563, 30.3429538], [53.9186792, 30.3442883], [53.9186319, 30.3445892], [53.9183447, 30.3443249], [53.9182293, 30.344293], [53.9175988, 30.3435681], [53.9174679, 30.3434176], [53.9175139, 30.3425837], [53.9187298, 30.3428209]], [[53.9543348, 30.3391294], [53.9548937, 30.3416706], [53.9527838, 30.3429697], [53.9526082, 30.3420819], [53.9523812, 30.3415725], [53.9522142, 30.3405536], [53.9543348, 30.3391294]]]}
{'name': '广州番禺职业技术学校', 'type': 'relation', 'nameisid': False, 'nodelist': [[[22.8987951, 113.3046961], [22.8989186, 113.3046547], [22.899099, 113.3046707], [22.8991842, 113.3046614], [22.8993084, 113.3046292], [22.899435, 113.3046225], [22.8995759, 113.3046345], [22.9001182, 113.3046587], [22.9003233, 113.3046493], [22.9004283, 113.3046681], [22.9005963, 113.3049309], [22.9006704, 113.3050328], [22.9008039, 113.3051455], [22.900836, 113.3053896]

{'name': 'Western Australian College of Agriculture - Cunderdin', 'type': 'way', 'nameisid': False, 'nodelist': [[-31.6239996, 117.2318854], [-31.6248333, 117.2264432], [-31.6250776, 117.2263842], [-31.626199, 117.2251665], [-31.6265565, 117.224213], [-31.6266307, 117.2242385], [-31.6271662, 117.2263185], [-31.6280946, 117.2258102], [-31.6281152, 117.2258907], [-31.6289031, 117.2256064], [-31.6289008, 117.2330307], [-31.6241412, 117.2321376], [-31.6239996, 117.2318854]]}
{'name': 'Central Community College - Grand Island Campus', 'type': 'way', 'nameisid': False, 'nodelist': [[40.8881005, -98.3769529], [40.8879708, -98.3686488], [40.8917179, -98.3686488], [40.8916692, -98.3765238], [40.8904851, -98.37691], [40.8881005, -98.3769529]]}
{'name': 'Bakersfield College', 'type': 'way', 'nameisid': False, 'nodelist': [[35.4049923, -118.9762016], [35.4049923, -118.9762027], [35.4050086, -118.9762501], [35.4050467, -118.9762694], [35.4059433, -118.9762792], [35.4088543, -118.9763111], [35.40927

{'name': 'Florida State College at Jacksonville - North Campus', 'type': 'way', 'nameisid': False, 'nodelist': [[30.4301553, -81.7268331], [30.4281756, -81.7268224], [30.4281007, -81.7264691], [30.4274899, -81.7264052], [30.4274896, -81.7253456], [30.4273689, -81.7253181], [30.427565, -81.7129505], [30.4283514, -81.7129505], [30.4288509, -81.7131222], [30.4291562, -81.7132831], [30.4294522, -81.7135943], [30.4309339, -81.7151784], [30.4309786, -81.7154018], [30.4308676, -81.7259641], [30.4301553, -81.7268331]]}
{'name': 'Florida State College at Jacksonville - Downtown Campus', 'type': 'way', 'nameisid': False, 'nodelist': [[30.33309, -81.6560709], [30.3363845, -81.6551222], [30.3365187, -81.6570856], [30.3364678, -81.6579707], [30.3370002, -81.6604273], [30.3372595, -81.6610925], [30.3376716, -81.6617523], [30.3378197, -81.662262], [30.3380883, -81.6634904], [30.3348985, -81.664397], [30.33309, -81.6560709]]}
{'name': 'Skidmore College', 'type': 'way', 'nameisid': False, 'nodelist': [

{'name': 'Agricultural Universiry, Shivamogga', 'type': 'way', 'nameisid': False, 'nodelist': [[13.9686573, 75.5803045], [13.9690113, 75.5784162], [13.9681544, 75.5781491], [13.968449, 75.5768498], [13.9700628, 75.5766352], [13.9703616, 75.5771084], [13.9712987, 75.5777843], [13.9716974, 75.5755409], [13.9749666, 75.5759486], [13.9746334, 75.5782338], [13.9768422, 75.578777], [13.9759296, 75.5815722], [13.9686573, 75.5803045]]}
{'name': '169375813', 'type': 'way', 'nameisid': True, 'nodelist': [[23.4312762, 85.3155779], [23.4345303, 85.3149568], [23.4343465, 85.318784], [23.4366262, 85.3187639], [23.4378396, 85.3222706], [23.4398067, 85.3235931], [23.4399722, 85.3270195], [23.4377661, 85.3271999], [23.4379499, 85.3312876], [23.4315336, 85.3315681], [23.4310555, 85.3215893], [23.4315703, 85.3191246], [23.4310923, 85.3188241], [23.4312762, 85.3155779]]}
{'name': '169375818', 'type': 'way', 'nameisid': True, 'nodelist': [[23.4324096, 85.3048584], [23.4353997, 85.3051134], [23.4355047, 85.

{'name': 'Lone Star College - CyFair', 'type': 'way', 'nameisid': False, 'nodelist': [[29.907534, -95.6861545], [29.9075265, -95.6864768], [29.9074158, -95.6912186], [29.9073829, -95.6926266], [29.9080446, -95.6926447], [29.9079246, -95.6932355], [29.9078362, -95.6942413], [29.9077804, -95.6972266], [29.9122537, -95.6973232], [29.9133189, -95.6973245], [29.9133408, -95.6960163], [29.913376, -95.6936501], [29.9134859, -95.6862671], [29.9127652, -95.68631], [29.9109773, -95.6862644], [29.9093428, -95.6862322], [29.9083292, -95.6862027], [29.907534, -95.6861545]]}
{'name': '江西电力职业技术学院', 'type': 'way', 'nameisid': False, 'nodelist': [[28.7234196, 115.7996643], [28.7219818, 115.7986705], [28.7205076, 115.7978779], [28.7196043, 115.797599], [28.719435, 115.797658], [28.719609, 115.7994336], [28.7202488, 115.8003616], [28.7203735, 115.8020943], [28.7206111, 115.8026737], [28.7204488, 115.8033335], [28.7203547, 115.8034864], [28.7178213, 115.8026013], [28.7173532, 115.8029258], [28.7170098, 11

{'name': 'Jordan Campus', 'type': 'way', 'nameisid': False, 'nodelist': [[40.5878633, -111.9770702], [40.5769058, -111.9769078], [40.5768931, -111.9721425], [40.5839641, -111.9721343], [40.5841133, -111.9724415], [40.5841816, -111.9723511], [40.5842152, -111.9722529], [40.5842297, -111.9718981], [40.5842612, -111.9716708], [40.5853698, -111.9717657], [40.5878435, -111.9717109], [40.5878816, -111.9721398], [40.5879259, -111.9737166], [40.5879149, -111.9755058], [40.5878828, -111.9762279], [40.5878633, -111.9770702]]}
{'name': 'Washtenaw Community College', 'type': 'way', 'nameisid': False, 'nodelist': [[42.2669262, -83.6683308], [42.2640795, -83.6607456], [42.2637804, -83.6602332], [42.2633851, -83.6598362], [42.2629899, -83.6596702], [42.2625412, -83.6596558], [42.2621246, -83.6597568], [42.2616599, -83.6600744], [42.2608801, -83.660926], [42.2602392, -83.6613157], [42.2594807, -83.6614023], [42.2593129, -83.668396], [42.2604344, -83.6691317], [42.2627769, -83.6702391], [42.2637766, -8

{'name': 'Bucks County Community College', 'type': 'way', 'nameisid': False, 'nodelist': [[40.2441088, -74.9689916], [40.2375865, -74.9581161], [40.2363922, -74.9591761], [40.2358599, -74.9609371], [40.2346819, -74.9658354], [40.2332985, -74.9659345], [40.2335723, -74.9706558], [40.2335441, -74.971235], [40.2329137, -74.9736875], [40.2329286, -74.9740966], [40.2332581, -74.9740175], [40.234008, -74.9735515], [40.2344502, -74.9732052], [40.234705, -74.9728085], [40.2349646, -74.9721914], [40.2352653, -74.9717714], [40.2355134, -74.9714597], [40.2359553, -74.9711124], [40.2364073, -74.970823], [40.2365977, -74.9707384], [40.236657, -74.9710181], [40.2367199, -74.9712679], [40.2386426, -74.9705892], [40.2402563, -74.9701607], [40.2404407, -74.9703088], [40.2405525, -74.9706178], [40.2406456, -74.9710813], [40.24066, -74.9717218], [40.2410676, -74.9718374], [40.2417081, -74.9713749], [40.2428605, -74.9702472], [40.2441088, -74.9689916]]}
{'name': 'Hartpury College', 'type': 'way', 'nameisi

{'name': '集宁师范学院(白海校区)', 'type': 'way', 'nameisid': False, 'nodelist': [[41.0300727, 113.2003492], [41.0300336, 113.2002973], [41.024696, 113.2013823], [41.02455, 113.2016091], [41.0249457, 113.2096161], [41.0249778, 113.2096757], [41.0313366, 113.20948], [41.0314022, 113.2093605], [41.0312588, 113.2087844], [41.0309759, 113.2057736], [41.0308119, 113.2058062], [41.0300727, 113.2003492]]}
{'name': '475603364', 'type': 'way', 'nameisid': True, 'nodelist': [[33.9389123, 116.4216403], [33.944419, 116.4214727], [33.9432231, 116.4308258], [33.9371045, 116.4285462], [33.9389123, 116.4216403]]}
{'name': '476752432', 'type': 'way', 'nameisid': True, 'nodelist': [[31.1355445, 118.6221765], [31.1358909, 118.6315636], [31.1415359, 118.6309971], [31.1412588, 118.6235926], [31.1390424, 118.6229857], [31.1368606, 118.6224192], [31.1355445, 118.6221765]]}
{'name': '重庆工程职业技术学院', 'type': 'way', 'nameisid': False, 'nodelist': [[29.3453018, 106.2470161], [29.3441712, 106.2518404], [29.3430159, 106.252303

{'name': 'Sussex County Community College', 'type': 'way', 'nameisid': False, 'nodelist': [[41.07371, -74.7557455], [41.069686, -74.7627568], [41.0680602, -74.7633255], [41.0668347, -74.7632611], [41.0661957, -74.7622848], [41.0663615, -74.7612816], [41.0662968, -74.7606754], [41.0658357, -74.7592861], [41.065314, -74.757328], [41.0650106, -74.7563088], [41.0649176, -74.755134], [41.069047, -74.753058], [41.0702765, -74.7524089], [41.0707537, -74.7533745], [41.0717809, -74.7539485], [41.0728364, -74.7544581], [41.0732691, -74.7550911], [41.07371, -74.7557455]]}
{'name': 'UIN Suska Riau', 'type': 'way', 'nameisid': False, 'nodelist': [[0.4691109, 101.352018], [0.4690679, 101.3587128], [0.4615912, 101.358737], [0.4616009, 101.3522326], [0.4691109, 101.352018]]}
{'name': 'Lone Star College - North Harris', 'type': 'way', 'nameisid': False, 'nodelist': [[30.0083795, -95.3803959], [30.0085977, -95.379996], [30.0087362, -95.3797452], [30.0085475, -95.379584], [30.0084654, -95.3793798], [30.0

In [11]:
def mergeways(alist):
    import copy
    finalnodelist = []
    for bound in alist:
        if bound['type'] == 'relation':
            nodelist = bound['nodelist']
            newnodelist = copy.deepcopy(nodelist)
            first = None
            last = None
            firstmembers = []
            lastmembers = []
            for i, waylist in enumerate(newnodelist):
                firstmembers.append(waylist[0])
                lastmembers.append(waylist[-1])
            
            while not firstmembers and not lastmembers:
                for first in firstmembers:
                    if firstmembers.count(first) > 1:
                        index1 = firstmembers.index(first)
                        index2 = [i for i, el in enumerate(firstmembers) if el == first][1]

                        #pop only the second firstmember since we're removing it
                        firstmembers.pop(index2)

                        #remove the 2nd way's first value
                        newnodelist[index2].pop(0)

                        #extend the lists with a copy to be safe
                        newnodelist[index1].extend(copy.deepcopy(newnodelist[index2]))

                        #remove the 2nd extra way
                        newnodelist.pop(index2)
                first = None
                last = None
                firstmembers = []
                lastmembers = []
                for i, waylist in enumerate(newnodelist):
                    firstmembers.append(waylist[0])
                    lastmembers.append(waylist[-1])
                for last in lastmembers:
                    if lastmembers.count(last) > 1:
                        index1 = lastmembers.index(last)
                        index2 = [i for i, el in enumerate(lastmembers) if el == last][1]

                        #pop only the second last member since we're removing it
                        lastmembers.pop(index2)

                        #remove and retain 2nd way's last member
                        retain = newnodelist[index2].pop(-1)

                        #remove 1st way's last member
                        newnodelist[index1].pop(-1)

                        #extend the two waylists with a copy to be safe
                        newnodelist[index1].extend(copy.deepcopy(newnodelist[index2]))

                        #add the shared last value
                        newnodelist[index1].append(retain)

                        #remove the extra way leftover
                        newnodelist.pop(index2)
                first = None
                last = None
                firstmembers = []
                lastmembers = []
                for i, waylist in enumerate(newnodelist):
                    firstmembers.append(waylist[0])
                    lastmembers.append(waylist[-1])
                
            first = None
            last = None
            firstmembers = []
            lastmembers = []
            for i, waylist in enumerate(newnodelist):
                firstmembers.append(waylist[0])
                lastmembers.append(waylist[-1])
            for first in firstmembers:
                while lastmembers.count(first) > 1:
                    index1 = firstmembers.index(first)
                    index2 = lastmembers.index(first)

                    firstmembers.pop(index1)
                    lastmembers.pop(index2)
                    #newnodelist[index1].extend(copy.deepcopy(newnodelist[index2]))
                    try: 
                        #extend the two lists
                        newnodelist[index1].extend(copy.deepcopy(newnodelist[index2]))
                    except IndexError as err:
                        print(newnodelist[59:62])
                        print(firstmembers)
                        print(lastmembers)
                        print(index1)
                        print(index2)
                        return
                    #pop the last shared value (shared at beggining and end)
                    newnodelist[index1].pop(-1)

                    #remove the extra way
                    newnodelist.pop(index2)
        finalnodelist.append(newnodelist)
    return finalnodelist

In [ ]:
for bound in bounding_university:
    newnodelist = bound['nodelist']
    firstmembers = []
    lastmembers = []
    for i, waylist in enumerate(newnodelist):
        firstmembers.append(waylist[0])
        lastmembers.append(waylist[-1])
    completion = False
    count = 0
    for first in firstmembers:
        if firstmembers.count(first) > 1:
            print(bound)
            print("")
    for last in lastmembers:
        if lastmembers.count(last) > 1:
            print(bound)
            print("")
    for last in lastmembers:
        if last in firstmembers:
            print(bound)
            print("")
#     count += len([print(["F"],first) for first in firstmembers if firstmembers.count(first) > 1])
#     count += len([print(["L"],last) for last in lastmembers if lastmembers.count(last) > 1])
#     count += len([print(last,first) for last in lastmembers for first in firstmembers if first == last])
#     print(count)

In [13]:
#there is ALWAYS a connected first and last member of same/different ways if firsts are connected
#or lasts are connected
#if the start or end of the way is the start or end of the next way they connect (including ending way to beginning)

In [ ]:
#         if firstlastcount != 0 and firstlastequalcount == 0:
#             print(bound)
#         if not completion:
#             if bound in seenbaditems:
#                 repeatbaditems.append(bound)
#             else:
#                 seenbaditems.append(bound)
                        
#     if bound['type'] == 'way':
#         waylist = bound['nodelist']
#         if waylist[0] != waylist[-1]:
#             print(bound)
#             if bound in seenbaditems:
#                 repeatbaditems.append(bound)
#             else:
#                 seenbaditems.append(bound)

In [28]:
import s2sphere as s2

In [ ]:
def createS2Mappings(searchlist):
    for bound in searchlist:
        flattened = list(arbitraryflatten(bound['nodelist']))
        latitudes = [element for i, element in enumerate(flattened) if i%2==0]
        longitudes = [element for i, element in enumerate(flattened) if i%2!=0]
        

In [52]:
with open(bounding_parent_global_university_path[:-5] + '_parsed.json', 'w') as write_file:
    json.dump(bounding_global_university_parsed, write_file, indent=4)
with open(bounding_parent_global_college_path[:-5] + '_parsed.json', 'w') as write_file:
    json.dump(bounding_global_college_parsed, write_file,indent=4)
with open(bounding_local_parent_path[:-5] + '_parsed.json', 'w') as write_file:
    json.dump(bounding_local_parsed, write_file,indent=4)